## CMI Detect Sleep States - Train

- Preprocess: https://www.kaggle.com/code/werus23/sleep-critical-point-prepare-data
- Train
- Infer

Reference:
- https://www.kaggle.com/code/werus23/sleep-critical-point-train/notebook


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sleep-critical-point-prepare-data/__results__.html
/kaggle/input/sleep-critical-point-prepare-data/__notebook__.ipynb
/kaggle/input/sleep-critical-point-prepare-data/__output__.json
/kaggle/input/sleep-critical-point-prepare-data/train_data.pkl
/kaggle/input/sleep-critical-point-prepare-data/custom.css
/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


## Configuration

In [2]:
import gc
import time
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os
import joblib
import random
import math
from math import pi, sqrt, exp

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from tqdm.auto import tqdm
import sklearn, sklearn.model_selection
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.metrics import average_precision_score
from timm.scheduler import CosineLRScheduler
plt.style.use("ggplot")

from pyarrow.parquet import ParquetFile
import pyarrow as pa
import ctypes

def normalize(y):
    mean = y[:, 0].mean().item()
    std = y[:, 0].std().item()
    y[:, 0] = (y[:, 0] - mean)/(std + 1e-16)
    
    mean = y[:, 1].mean().item()
    std = y[:, 1].std().item()
    y[:, 1] = (y[:, 1] - mean)/(std + 1e-16)
    
    return y

device = "cuda" if torch.cuda.is_available() else "cpu"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
EPOCHS = 5
WARMUP_PROP = 0.2
BATCH_SIZE = 1
WORKERS = 4
TRAIN_PROP = 0.9
max_chunk_size = 150000
if device == "cpu":
    torch.set_num_interop_threads(WORKERS)
    torch.set_num_threads(WORKERS)

In [4]:
def plot_history(history, model_path=".", show=True):
    # Loss vs Epochs
    epochs = range(1, len(history["train_loss"]) + 1)
    
    plt.figure()
    plt.plot(epochs, history["train_loss"], label="Training Loss")
    plt.plot(epochs, history["valid_loss"], label="Validation Loss")
    
    plt.title("Loss vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabal("Loss")
    plt.legend()
    
    plt.savefig(os.path.join(modle_path, "loss_epochs.png"))
    if show:
        plt.show()  
    plt.close()
    
    # LR vs Epochs
    plt.figure()
    plt.plot(epochs, history["lr"])
    
    plt.title("Learning rate vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("LR")
    plt.savefig(os.path.join(model_path, "lr_epochs.png"))
    if show:
        plt.show()
        
    plt.close()

## Dataset

In [5]:
SIGMA = 720
SAMPLE_FREQ = 12

class SleepDataset(Dataset):
    def __init__(self, file):
        self.targets, self.data, self.ids = joblib.load(file)
    
    def downsample_seq_generate_features(self, feat, downsample_factor=SAMPLE_FREQ):
        if len(feat) % SAMPLE_FREQ == 0:
            feat = np.concatenate([feat, np.zeros(SAMPLE_FREQ - len(feat) % SAMPLE_FREQ)])
        
        # generate seq
        feat = np.reshape(feat, (-1, SAMPLE_FREQ))
        feat_mean = np.mean(feat, 1)
        feat_std = np.std(feat, 1)
        feat_median = np.median(feat, 1)
        feat_max = np.max(feat, 1)
        feat_min = np.min(feat, 1)
        
        return np.dstack([feat_mean, feat_std, feat_median, feat_max, feat_min])[0]
    
    def downsample_seq(self, feat, downsample_feactor=SAMPLE_FREQ):
        if len(feat) % SAMPLE_FREQ == 0:
            feat = np.concatenate([
                feat,
                np.zeros(SAMPLE_FREQ - len(feat) % SAMPLE_FREQ) + feat[-1],
            ])
            
        feat = np.reshape(feat, (-1, SAMPLE_FREQ))
        feat_mean = np.mean(feat, 1)

        return feat_mean
        
    def gauss(self, n=SIGMA, sigma=SIGMA*0.15):
        r = range(-int(n/2), int(n/2)+1)
        
        return [1 / (sigma * sqrt(2*pi)) * exp(-float(x)**2/(2*sigma**2)) for x in r]
    
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        X = self.data[idx][["anglez", "enmo"]]
        y = self.targets[idx]
        
        # turn target inds into array
        target_guassian = np.zeros((len(X), 2))
        for s, e in y:
            st1, st2 = max(0, s-SIGMA//2), s+SIGMA//2+1
            ed1, ed2 = e-SIGMA//2, min(len(X), e+SIGMA//2+1)
            
            target_guassian[st1:st2,0] = self.gauss()[st1-(s-SIGMA//2):]
            target_guassian[ed1:ed2,1] = self.gauss()[:SIGMA+1-((e+SIGMA//2+1)-ed2)]
            
            gc.collect()
        
        y = target_guassian
        gc.collect()
        
        X = np.concatenate(
            [
                self.downsample_seq_generate_features(
                    X.values[:,i],
                    SAMPLE_FREQ,
                ) for i in range(X.shape[1])
            ],
            -1,
        )
        gc.collect()
        
        y = np.dstack(
            [
                self.downsample_seq(
                    y[:,i],
                    SAMPLE_FREQ,
                ) for i in range(y.shape[1])
            ]
        )[0]
            
        gc.collect()
        
        y = normalize(torch.from_numpy(y))
        X = torch.from_numpy(X)
        
        return X, y
    
train_ds = SleepDataset("/kaggle/input/sleep-critical-point-prepare-data/train_data.pkl")

## Model

In [6]:
class ResidualBiGRU(nn.Module):
    """
    残差接続を行う双方向GRU
    """
    def __init__(self, hidden_size, n_layers=1, bidir=True):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        dir_factor = 2 if bidir else 1
        
        # @see: https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
        self.gru = nn.GRU(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=n_layers,
            bias=True, # use bias weights
            batch_first=True, # (seq, batch,feature)を(batch, seq, feature)へ
            dropout=0,
            bidirectional=True,
        )
        self.fc1 = nn.Linear(
            in_features=hidden_size*dir_factor,
            out_features=hidden_size*dir_factor*2,
        )
        self.ln1 = nn.LayerNorm(normalized_shape=hidden_size*dir_factor*2)
        self.fc2 = nn.Linear(
            in_features=hidden_size*dir_factor*2,
            out_features=hidden_size,
        )
        self.ln2 = nn.LayerNorm(normalized_shape=hidden_size)
        
    def forward(self, x, h=None):
        """順伝播"""
        output, new_h = self.gru(x, h)
        
        output = self.fc1(output)
        output = self.ln1(output)
        output = F.relu(output)
        
        output = self.fc2(output)
        output = self.ln2(output)
        output = F.relu(output)
        
        # skip connection
        output = output + x
        
        return output, new_h
        
class MultiResidualBiGRU(nn.Module):
    """
    ResidulaBiGRUをn_layers分だけ重ねる
    """
    def __init__(self, input_size, hidden_size, out_size, n_layers, bidir=True):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.out_size = out_size
        self.n_layers = n_layers
        
        self.fc_in = nn.Linear(
            in_features=input_size,
            out_features=hidden_size,
        )
        self.ln = nn.LayerNorm(normalized_shape=hidden_size)
        self.bigrus = nn.ModuleList(
            [
                ResidualBiGRU(
                    hidden_size=hidden_size,
                    n_layers=1,
                    bidir=bidir,
                )
                for _ in range(n_layers)
            ]
        )
        self.fc_out = nn.Linear(
            in_features=hidden_size,
            out_features=out_size,
        )
        
    def forward(self, x, h=None):
        """順伝播"""
        if h is None:
            # (re)initialize the hidden state
            h = [None for _ in range(self.n_layers)]
            
        x = self.fc_in(x)
        x = self.ln(x)
        x = F.relu(x)
        
        new_h = []
        for i, bigru in enumerate(self.bigrus):
            x, new_hi = bigru(x, h[i])
            new_h.append(new_hi)
            
        x = self.fc_out(x)
        
        return x, new_h # log probabilities + hidden states

## Train and Eval

In [7]:
train_size = int(TRAIN_PROP * len(train_ds))
valid_size = len(train_ds) - train_size
indices = torch.randperm(len(train_ds))

train_sampler = SubsetRandomSampler(indices[:train_size])
valid_sampler = SubsetRandomSampler(indices[train_size: train_size+valid_size])

steps = train_size * EPOCHS
warmup_steps = int(steps * WARMUP_PROP)

# モデルインスタンスの生成
model = MultiResidualBiGRU(
    input_size=10,
    hidden_size=64,
    out_size=2,
    n_layers=5,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0)
scheduler = CosineLRScheduler(
    optimizer,
    t_initial=steps,
    warmup_t=warmup_steps,
    warmup_lr_init=1e-6,
    lr_min=2e-8,
)

dt = time.time()

model_path="/kaggle/working/"
history = {
    "train_loss": [],
    "valid_loss": [],
    "valid_mAP": [],
    "lr": [],
}

best_valid_loss = np.inf
criterion = torch.nn.MSELoss()

In [8]:
def evaluate(
    model: nn.Module,
    max_chunk_size: int,
    loader: DataLoader,
    device,
    criterion,
):
    model.eval()
    valid_loss = 0.0
    y_true_full = torch.FloatTensor([]).half()
    y_pred_full = torch.FloatTensor([]).half()
    for X_batch, y_batch in tqdm(loader, desc="Eval", unit="batch"):
        # シーケンスが長すぎるのでchunk-based手法を用いる
        y_batch = y_batch.to(device, non_blocking=True)
        pred = torch.zeros(y_batch.shape).to(device, non_blocking=True).half()
        
        # (re)initialize model's hidden state(s)
        h = None
        
        seq_len = X_batch.shape[1]
        for i in range(0, seq_len, max_chunk_size):
            X_chunk = X_batch[:, i: i+max_chunk_size].float().to(device, non_blocking=True)
            y_pred, h = model(X_chunk, h)
            h = [hi.detach() for hi in h]
            pred[:, i: i+max_chunk_size] = y_pred.half()
            
            del X_chunk
            gc.collect()
            
        loss = criterion(pred.float(), y_batch.float())
        valid_loss += loss.item()
        
        del pred, loss
        gc.collect()
    
    valid_loss /= len(loader)
    
    y_true_full = y_true_full.squeeze(0)
    y_pred_full = y_pred_full.squeeze(0)
    
    gc.collect()
    
    return valid_loss

In [9]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=BATCH_SIZE,
    sampler=train_sampler,
    # kaggle notebookのGPU環境について調べる
    pin_memory=True,
    num_workers=1,
)
valid_loader = DataLoader(
    dataset=train_ds,
    batch_size=1,
    sampler=valid_sampler,
    pin_memory=True,
    num_workers=1,
)

for epoch in range(1, EPOCHS+1):
    train_loss = 0.0
    n_tot_chunks = 0
    pbar = tqdm(train_loader, desc="Training", unit="batch")
    
    model.train()
    for step, (X_batch, y_batch) in enumerate(pbar):
        y_batch = y_batch.to(device, non_blocking=True)
        pred = torch.zeros(y_batch.shape).to(device, non_blocking=True)
        optimizer.zero_grad()
        scheduler.step(step+train_size*epoch)
        h = None
        
        seq_len = X_batch.shape[1]
        for i in range(0, seq_len, max_chunk_size):
            X_chunk = X_batch[:, i:i+max_chunk_size].float()
            
            X_chunk = X_chunk.to(device, non_blocking=True)
            
            y_pred, h = model(X_chunk, h)
            
            h = [hi.detach() for hi in h]
            pred[:, i:i+max_chunk_size] = y_pred
            
            del X_chunk, y_pred
        
        loss = criterion(
            normalize(pred).float(),
            y_batch.float(),
        )
        loss.backward()
        
        train_loss += loss.item()
        n_tot_chunks += 1
        
        pbar.set_description(f"Training: loss = {(train_loss/n_tot_chunks):.2f}")
        
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1e-1)
        optimizer.step()
        
        del pred, loss, y_batch, X_batch, h
        gc.collect()
        
    train_loss /= len(train_loader)
    del pbar
    gc.collect()
    
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    
    if epoch % 1 == 0:
        valid_loss = evaluate(
            model,
            max_chunk_size,
            valid_loader,
            device,
            criterion,
        )
        
        history["train_loss"].append(train_loss)
        history["valid_loss"].append(valid_loss)
        history["lr"].append(optimizer.param_groups[0]["lr"])
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(
                model.state_dict(),
                os.path.join(model_path, "model_best.pth")
            )
            
        dt = time.time() - dt
        
        print(
            f"{epoch}/{EPOCHS} -- ",
            f"train_loss = {train_loss:.6f} -- ",
            f"valid_loss = {valid_loss:.6f} -- ",
            f"time = {dt:.6f}s",
        )
        
        dt = time.time()

Training:   0%|          | 0/249 [00:00<?, ?batch/s]

Eval:   0%|          | 0/28 [00:00<?, ?batch/s]

1/5 --  train_loss = 0.715468 --  valid_loss = 0.504810 --  time = 761.558617s


Training:   0%|          | 0/249 [00:00<?, ?batch/s]

Eval:   0%|          | 0/28 [00:00<?, ?batch/s]

...

Eval:   0%|          | 0/28 [00:00<?, ?batch/s]

4/5 --  train_loss = 0.364887 --  valid_loss = 0.356822 --  time = 776.576065s


Training:   0%|          | 0/249 [00:00<?, ?batch/s]

Eval:   0%|          | 0/28 [00:00<?, ?batch/s]

5/5 --  train_loss = 0.358690 --  valid_loss = 0.356823 --  time = 775.639050s


In [10]:
# # plot
# plot_history(history, model_path=model_path)
# history_path = os.path.join(model_path, "history.json")
# with open(history_path, "w", encoding="utf-8") as f:
#     json.dump(history, f, ensure_ascii=False, indent=4)